In [ ]:
from braivest.utils import load_wandb_model
from braivest.model.emgVAE import emgVAE
from braivest.preprocess.dataset_utils import load_data, bin_data
from braivest.analysis.plotting_utils import *
import plotly.express as px
import wandb
import tensorflow as tf
import matplotlib.pyplot as plt

from scipy.signal import welch
from scipy.stats import zscore

In [ ]:
#lfp =
#emg = 
binned_lfp = bin_data(lfp, original_sample=1000, sampling_rate=0.5)

In [ ]:
#val_data_artifact_dir = 
subject0_sess = load_data(artifact_dir, 'subject0_vis11_val.npy', allow_pickle=True)

In [ ]:
model = load_wandb_model("juliahwang/lfp_VAE/v2l9tltt")
encodings = tf.convert_to_tensor(model.encode(subject0_sess[0][0])).numpy()

In [ ]:
f, Pxx = welch(binned_lfp, fs=1000)

In [ ]:
# Let's see what frequencies there are:
f

In [ ]:
BAND = 'delta' #change this manually
bands = {'delta': (0, 2), 'theta': (2, 4), 'beta': (4, 8), 'gamma': (8, 13)}
start, stop = bands[BAND]
color = get_feature_color(Pxx, f, start, stop)
fig = plot_encodings(encodings, color=color, x_range = (-6, 3), scatter_kwargs={'color_continuous_scale': 'portland', 'range_color': (-3, 3)})
fig.show()